In [1]:
# !pip install numpy
# !pip install tensorflow
# !pip install pillow


     ---------------------------------------- 14.9/14.9 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 272.8/272.8 MB 1.9 MB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl (440 kB)
     ---------------------------------------- 4.1/4.1 MB 3.1 MB/s eta 0:00:00
  Using cached libclang-16.0.0-py2.py3-none-win_amd64.whl (24.4 MB)
     -------------------------------------- 420.6/420.6 kB 4.4 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 3.0 MB/s eta 0:00:00
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Using cached wrapt-1.14.1-cp38-cp38-win_amd64.whl (35 kB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached jax-0.4.8.tar.gz (1.2 MB)
  Installing build dependencies: started
  Installing build dependencies: finishe

In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# All images will be rescaled by 1./255
trin_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
# Flow training images in batches of 128 using train_datagen generator
train_generator = trin_datagen.flow_from_directory(
        'data/train',  # This is the source directory for training images
        target_size=(48, 48),  # All images will be resized to 48x48
        batch_size=64, 
        color_mode="grayscale",
        class_mode='categorical')

# Flow validation images in batches of 128 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.optimizers import Adam

In [5]:
emotion_model = Sequential()

In [6]:
# 1st convolution layer
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# 2nd convolution layer
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

# 3rd convolution layer
# emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
# emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
# emotion_model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
# emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
# emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())

# Fully connected layer 1st layer
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))

emotion_model.add(Dense(7, activation='softmax')) # 7 because we have 7 classes {0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral}

emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

c:\Users\User\anaconda3\envs\fyp_emotion_detection\Lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [7]:
# emotion_model_info = emotion_model.fit_generator(
#         train_generator,
#         steps_per_epoch=28709 // 64,
#         epochs=50,
#         validation_data=validation_generator,
#         validation_steps=7178 // 64)

# training neural network
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/50
448/448 [==============================] - 373s 831ms/step - loss: 1.8027 - accuracy: 0.2606 - val_loss: 1.7121 - val_accuracy: 0.3341
Epoch 2/50
448/448 [==============================] - 205s 457ms/step - loss: 1.6313 - accuracy: 0.3643 - val_loss: 1.5468 - val_accuracy: 0.4092
Epoch 3/50
448/448 [==============================] - 213s 476ms/step - loss: 1.5285 - accuracy: 0.4143 - val_loss: 1.4632 - val_accuracy: 0.4474
Epoch 4/50
448/448 [==============================] - 212s 474ms/step - loss: 1.4544 - accuracy: 0.4432 - val_loss: 1.3956 - val_accuracy: 0.4724
Epoch 5/50
448/448 [==============================] - 216s 482ms/step - loss: 1.3896 - accuracy: 0.4704 - val_loss: 1.3514 - val_accuracy: 0.4851
Epoch 6/50
448/448 [==============================] - 219s 489ms/step - loss: 1.3374 - accuracy: 0.4916 - val_loss: 1.2939 - val_accuracy: 0.5106
Epoch 7/50
448/448 [==============================] - 223s 497ms/step - loss: 1.2970 - accuracy: 0.5089 - val_loss: 1.2718 -

In [8]:
# Save model
model_json = emotion_model.to_json()
with open("model/emotion_model.json", "w") as json_file:
    json_file.write(model_json)

In [9]:
# Save weights
emotion_model.save_weights('model/emotion_model.h5')

In [10]:
# evaluate mmodel
scores = emotion_model.evaluate(validation_generator, verbose=0)
print('Test accuracy', scores[1])

Test accuracy 0.6251044869422913


In [11]:
from keras.utils.vis_utils import plot_model

In [12]:
plot_model(emotion_model, to_file='model/model_plot.png', show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
